In [1]:
import pandas as pd
print("version {}".format(pd.__version__))

version 1.2.4


## Load Data

In [2]:
rents = pd.read_json("data/rents.json")
rents.head(2)
# rents.columns

,bedrooms,type,location,price
0,4,Apartment / Flat,Lavington,75000
1,4,House,Kitisuru,300000


## Preprocess Data

### Convert to required types

In [3]:
rents["price"] = pd.to_numeric(rents["price"])
rents["bedrooms"] = pd.to_numeric(rents["bedrooms"])

print("Shape: ",rents.shape) # 101701 Values
rents.info()

Shape:  (101701, 4)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101701 entries, 0 to 101700
Data columns (total 4 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   bedrooms  100699 non-null  float64
 1   type      101701 non-null  object 
 2   location  101701 non-null  object 
 3   price     101649 non-null  float64
dtypes: float64(2), object(2)
memory usage: 3.1+ MB


### Drop rows with bad data

#### Null

In [4]:
rents[rents.isnull().values].shape # 1054 N/A values

(1054, 4)

In [5]:
rents.dropna(inplace=True)
rents.shape # 100648

(100648, 4)

#### Price > 1M

In [6]:
rents[rents["price"] > 1000000].shape # 543 records

(543, 4)

In [7]:
rents.drop(rents[rents['price'] > 1000000].index, inplace=True)
rents.shape # 100105

(100105, 4)

#### Price < 1k

In [8]:
rents[rents["price"] < 1000].shape # 63

(63, 4)

In [9]:
rents.drop(rents[rents['price'] < 1000].index, inplace=True)
rents.shape # 100042

(100042, 4)

#### Bedrooms > 10

In [10]:
rents[rents["bedrooms"] > 10].shape # 97 records

(97, 4)

In [11]:
rents.drop(rents[rents['bedrooms'] > 10].index, inplace=True)
rents.shape # 99945

(99945, 4)

####  Bedrooms < 1

In [12]:
rents.drop(rents[rents['bedrooms'] < 1].index, inplace=True)
rents.shape # 99945

(99945, 4)

#### Duplicates

In [13]:
rents.drop_duplicates(inplace=True)
rents.shape # 8773 clean data

(8773, 4)

### Encoding Data

#### Rename to 1-word values

In [17]:
rents.replace('Apartment / Flat', 'Apartment', inplace=True)
rents

,bedrooms,type,location,price
0,4.0,Apartment,Lavington,75000.0
1,4.0,House,Kitisuru,300000.0
2,5.0,Townhouse,Lavington,300000.0
3,2.0,Apartment,Lavington,180000.0
4,4.0,Townhouse,Lavington,180000.0
...,...,...,...,...
101633,5.0,House,Runda,535353.0
101647,3.0,House,Nairobi West,55000.0
101665,7.0,House,Nyari,200000.0
101671,2.0,Townhouse,Kyuna,150000.0


#### Output non-encoded to file

In [32]:
rents['location'].unique()
# rents.to_csv("data/rents_not_encoded_clean.csv", index=False)

array(['Lavington', 'Kitisuru', 'Westlands', 'Langata', 'Runda',
       'Spring Valley', 'Peponi', 'Windsor', 'Karen', 'Thigiri',
       'Kilimani', 'Kiamumbi', 'Kileleshwa', 'Riverside', 'Ngara',
       'Loresho', 'Rosslyn', 'Parklands', 'New Muthaiga', 'Imara Daima',
       'Ongata Rongai', 'Nyari', 'Gigiri', 'Lower Kabete',
       'Garden Estate', 'Ridgeways', 'Muthaiga', 'Redhill',
       'Muthaiga North', 'Lakeview', 'Upper Hill', 'Kiambu Road', 'Kyuna',
       'Ngong Rd', 'Hurlingham', 'Muthangari', 'Uthiru', 'Riruta',
       'Valley Arcade', 'South C', 'Kahawa sukari', 'Mountain View',
       'Kabete', 'Ruaka', 'Ruaraka', 'Dagoretti', 'Syokimau',
       'Nairobi CBD', 'South B', 'Thome', 'Milimani', 'Kahawa West',
       'Mombasa Rd', 'Kerarapon', 'Nairobi West', 'Mlolongo', 'Hillview',
       'Pangani', 'Saika', 'Roysambu', 'Madaraka', 'Katani', 'Embakasi',
       'Mbagathi Way', 'Kasarani', 'Ngumo estate', 'Donholm', 'Woodley',
       'Hardy', 'Muthurwa', 'Lenana', 'Joska', 'K

#### Hot Encode

In [33]:
loc = pd.get_dummies(rents.location, prefix="location")
typ = pd.get_dummies(rents.type, prefix="type")
loc.head(2)

,location_Allsops,location_Baba Dogo,location_Balozi,location_Buruburu,location_Dagoretti,location_Donholm,location_Embakasi,location_Garden Estate,location_Gigiri,location_Githurai,...,location_Thome,location_Umoja,location_Upper Hill,location_Utawala,location_Uthiru,location_Valley Arcade,location_Waithaka,location_Westlands,location_Windsor,location_Woodley
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### Concatenate encoded data to df

In [39]:
rents = pd.concat([rents, typ, loc], axis=1)
rents.drop(['location', 'type'], axis=1, inplace=True)


In [40]:
rents.head(2)

,bedrooms,price,type_Apartment,type_House,type_Townhouse,location_Allsops,location_Baba Dogo,location_Balozi,location_Buruburu,location_Dagoretti,...,location_Thome,location_Umoja,location_Upper Hill,location_Utawala,location_Uthiru,location_Valley Arcade,location_Waithaka,location_Westlands,location_Windsor,location_Woodley
0,4.0,75000.0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,4.0,300000.0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Export cleaned data to json,csv

In [41]:
rents.to_json("data/rents_clean.json", orient='records')
rents.to_csv("data/rents_clean.csv", index=False)
rents.to_excel("data/rents_clean.xlsx", sheet_name="rents_clean", index=False)

In [42]:
rents.loc[11]

bedrooms                       5.0
price                     230000.0
type_Apartment                 0.0
type_House                     1.0
type_Townhouse                 0.0
                            ...   
location_Valley Arcade         0.0
location_Waithaka              0.0
location_Westlands             0.0
location_Windsor               0.0
location_Woodley               0.0
Name: 11, Length: 106, dtype: float64